In [5]:
import torch
from torch import nn
import numpy as np
import cv2
import ipywidgets as widgets
import asyncio
import matplotlib.pyplot as plt
import skimage

torch.set_grad_enabled(False)

In [6]:
#screen_h, screen_w = 1440, 2560
#screen_h, screen_w = 1440//4, 2560//4
screen_h, screen_w = 200, 200


if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU instead")


def NormalizeEnergy(energy, width=3):
    mean = torch.mean(energy.flatten())
    std = torch.std(energy.flatten())
    h_cutoff = mean + std*width
    energy_normed = (torch.clip(energy, 0, h_cutoff))/(h_cutoff)
    return energy_normed

def NormalizeSine(arr):
    return (arr + 1.0)/2.0
    
def Colorize(arr):
    '''
    Assumes a tensor of shape (h, w) and outputs 
    (h, w, (rgb))
    '''
    # First convert to CIELAB space
    lab = np.stack([60*np.ones_like(arr), 128*np.cos(arr), 128*np.sin(arr)], axis=-1)
    rgb = skimage.color.lab2rgb(lab)
    return rgb
    

GPU is available


In [7]:
# RGB --> Init Phase, Coupling Constants foreach scale
# (H, W, 3) --> (H, W, 1), (H, W, N_convs)

class AutoEncoder(nn.Module):
    def __init__(self, grid, input_chans=3, num_scales=3):
        super(AutoEncoder, self).__init__()
        self.em_grid = grid
        self.num_em_steps = num_em_steps
        ic = input_chans
        self.num_scales = num_scales
        self.bypass_em = bypass_em
        # Convolutions for common feature extractor
        self.conv1 = nn.Conv2d(ic,  8, kernel_size=5, stride=1, padding='same')
        self.conv2 = nn.Conv2d( 8, 16, kernel_size=5, stride=1, padding='same')
        self.conv3 = nn.Conv2d(16,  8, kernel_size=5, stride=1, padding='same')
        self.conv4 = nn.Conv2d( 8,  8, kernel_size=5, stride=1, padding='same')
        self.conv5 = nn.Conv2d( 8,  8, kernel_size=5, stride=1, padding='same')
        self.conv6 = nn.Conv2d( 8,  8, kernel_size=5, stride=1, padding='same')
        self.conv_phases = nn.Conv2d( 8,  1, kernel_size=5, stride=1, padding='same')
        self.conv_couple = nn.Conv2d( 8,  self.num_scales, kernel_size=5, stride=1, padding='same')
        
    def forward(self, x):
        ## 1 - Extract features
        # Convert image into amplitude, frequency, and phase shift for our CCs.
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.conv2(x)
        x = torch.relu(x)
        x = self.conv3(x)
        x = torch.relu(x)
        x = self.conv4(x)
        x = torch.relu(x)
        x = self.conv5(x)
        x = torch.relu(x)
        x = self.conv6(x)
        x = torch.relu(x)
        z_phases = self.conv_phases(x)
        z_couple = self.conv_couple(x)
        
        return z_phases, z_couple
        

In [9]:
# Instantiate convolution dict
coupling_convs = {}
num_scales = 5
size = 3
grow = 3
for i in range(num_scales):
    conv = torch.nn.Conv2d(1, 1, size, bias=False, padding='same', padding_mode='circular', device=device)
    if(i == 0):
        k = torch.ones((size, size)).to(device)
        k = k /(torch.sum(k) - 1.0)
    else:
        k = torch.zeros((last_size, last_size)).to(device)
        k = torch.nn.functional.pad(k, (grow, grow, grow, grow), value=1.0)
        k = k /(torch.sum(k))
    
    
    k[k.shape[0]//2, k.shape[0]//2] = -1
    print(size, k.shape, conv.weight.shape, k.sum())
    conv.weight = torch.nn.Parameter(k[None, None, ...])
    coupling_convs[k.shape[0]] = conv
    last_size = size
    size += grow*2

3 torch.Size([3, 3]) torch.Size([1, 1, 3, 3]) tensor(0., device='cuda:0')
9 torch.Size([9, 9]) torch.Size([1, 1, 9, 9]) tensor(5.9605e-08, device='cuda:0')
15 torch.Size([15, 15]) torch.Size([1, 1, 15, 15]) tensor(-5.9605e-08, device='cuda:0')
21 torch.Size([21, 21]) torch.Size([1, 1, 21, 21]) tensor(0., device='cuda:0')
27 torch.Size([27, 27]) torch.Size([1, 1, 27, 27]) tensor(5.9605e-08, device='cuda:0')


In [32]:
def run_sim(phases, coupling_constants, coupling_convs, frequency = 0.1, num_steps = 2000):
    img_mean = torch.zeros_like(phases).cuda()
    for t in range(num_steps):
        # Calculate the oscillator's positions
        img_out = torch.sin(frequency*t + phases)
        img_mean += img_out
        # Calculate the errors
        error_sum = torch.zeros_like(img_out)
        for k_idx, k_key in enumerate(coupling_convs):
            error = torch.sin(coupling_convs[k_key](phases))
            error_sum -= error*coupling_constants[..., k_idx]
            print(t, torch.abs(error_sum).mean())
        phases += error_sum

        #print(img_out[0,0,...].shape, Colorize(img_out[0,0,...].cpu().detach().numpy()).shape)
        if((t % 10) == 0):
            img_out_waves = Colorize(img_out[0,0,...].cpu().detach().numpy())
            img_out_phases = Colorize(phases[0,0,...].cpu().detach().numpy())
            cv2.imshow('Waves', img_out_waves)
            cv2.imshow('Phases', img_out_phases)
            cv2.waitKey(1)
    return img_mean / num_steps


run_sim(torch.rand((1, 1, screen_h, screen_w)).cuda(), torch.ones((1, 1, screen_h, screen_w, num_scales)).cuda(), coupling_convs)


0 tensor(0.2544, device='cuda:0')
0 tensor(0.4943, device='cuda:0')
0 tensor(0.7372, device='cuda:0')
0 tensor(0.9807, device='cuda:0')
0 tensor(1.2247, device='cuda:0')
1 tensor(0.6466, device='cuda:0')
1 tensor(1.2616, device='cuda:0')
1 tensor(1.9184, device='cuda:0')
1 tensor(2.5854, device='cuda:0')
1 tensor(3.2571, device='cuda:0')
2 tensor(0.6387, device='cuda:0')
2 tensor(0.8400, device='cuda:0')
2 tensor(1.2241, device='cuda:0')
2 tensor(1.6662, device='cuda:0')
2 tensor(2.1354, device='cuda:0')
3 tensor(0.6301, device='cuda:0')
3 tensor(0.9034, device='cuda:0')
3 tensor(1.4158, device='cuda:0')
3 tensor(1.9921, device='cuda:0')
3 tensor(2.5922, device='cuda:0')
4 tensor(0.6341, device='cuda:0')
4 tensor(0.8355, device='cuda:0')
4 tensor(1.3110, device='cuda:0')
4 tensor(1.8708, device='cuda:0')
4 tensor(2.4620, device='cuda:0')
5 tensor(0.6343, device='cuda:0')
5 tensor(0.8600, device='cuda:0')
5 tensor(1.3731, device='cuda:0')
5 tensor(1.9595, device='cuda:0')
5 tensor(2.576

101 tensor(0.6397, device='cuda:0')
101 tensor(0.8135, device='cuda:0')
101 tensor(1.2189, device='cuda:0')
101 tensor(1.7265, device='cuda:0')
101 tensor(2.2846, device='cuda:0')
102 tensor(0.6398, device='cuda:0')
102 tensor(0.8142, device='cuda:0')
102 tensor(1.2160, device='cuda:0')
102 tensor(1.7236, device='cuda:0')
102 tensor(2.2831, device='cuda:0')
103 tensor(0.6380, device='cuda:0')
103 tensor(0.8150, device='cuda:0')
103 tensor(1.2172, device='cuda:0')
103 tensor(1.7262, device='cuda:0')
103 tensor(2.2824, device='cuda:0')
104 tensor(0.6398, device='cuda:0')
104 tensor(0.8160, device='cuda:0')
104 tensor(1.2174, device='cuda:0')
104 tensor(1.7225, device='cuda:0')
104 tensor(2.2799, device='cuda:0')
105 tensor(0.6398, device='cuda:0')
105 tensor(0.8149, device='cuda:0')
105 tensor(1.2089, device='cuda:0')
105 tensor(1.7109, device='cuda:0')
105 tensor(2.2607, device='cuda:0')
106 tensor(0.6430, device='cuda:0')
106 tensor(0.8191, device='cuda:0')
106 tensor(1.2187, device='c

161 tensor(0.6449, device='cuda:0')
161 tensor(0.8287, device='cuda:0')
161 tensor(1.1944, device='cuda:0')
161 tensor(1.6584, device='cuda:0')
161 tensor(2.1703, device='cuda:0')
162 tensor(0.6427, device='cuda:0')
162 tensor(0.8293, device='cuda:0')
162 tensor(1.1983, device='cuda:0')
162 tensor(1.6569, device='cuda:0')
162 tensor(2.1690, device='cuda:0')
163 tensor(0.6443, device='cuda:0')
163 tensor(0.8280, device='cuda:0')
163 tensor(1.1965, device='cuda:0')
163 tensor(1.6589, device='cuda:0')
163 tensor(2.1667, device='cuda:0')
164 tensor(0.6459, device='cuda:0')
164 tensor(0.8279, device='cuda:0')
164 tensor(1.1948, device='cuda:0')
164 tensor(1.6536, device='cuda:0')
164 tensor(2.1633, device='cuda:0')
165 tensor(0.6452, device='cuda:0')
165 tensor(0.8298, device='cuda:0')
165 tensor(1.1991, device='cuda:0')
165 tensor(1.6598, device='cuda:0')
165 tensor(2.1681, device='cuda:0')
166 tensor(0.6455, device='cuda:0')
166 tensor(0.8287, device='cuda:0')
166 tensor(1.1953, device='c

223 tensor(2.0897, device='cuda:0')
224 tensor(0.6475, device='cuda:0')
224 tensor(0.8294, device='cuda:0')
224 tensor(1.1818, device='cuda:0')
224 tensor(1.6181, device='cuda:0')
224 tensor(2.0988, device='cuda:0')
225 tensor(0.6474, device='cuda:0')
225 tensor(0.8251, device='cuda:0')
225 tensor(1.1738, device='cuda:0')
225 tensor(1.6085, device='cuda:0')
225 tensor(2.0933, device='cuda:0')
226 tensor(0.6470, device='cuda:0')
226 tensor(0.8275, device='cuda:0')
226 tensor(1.1809, device='cuda:0')
226 tensor(1.6179, device='cuda:0')
226 tensor(2.1012, device='cuda:0')
227 tensor(0.6474, device='cuda:0')
227 tensor(0.8295, device='cuda:0')
227 tensor(1.1740, device='cuda:0')
227 tensor(1.6046, device='cuda:0')
227 tensor(2.0864, device='cuda:0')
228 tensor(0.6483, device='cuda:0')
228 tensor(0.8239, device='cuda:0')
228 tensor(1.1731, device='cuda:0')
228 tensor(1.6046, device='cuda:0')
228 tensor(2.0804, device='cuda:0')
229 tensor(0.6478, device='cuda:0')
229 tensor(0.8285, device='c

287 tensor(1.1631, device='cuda:0')
287 tensor(1.5714, device='cuda:0')
287 tensor(2.0212, device='cuda:0')
288 tensor(0.6509, device='cuda:0')
288 tensor(0.8326, device='cuda:0')
288 tensor(1.1597, device='cuda:0')
288 tensor(1.5654, device='cuda:0')
288 tensor(2.0171, device='cuda:0')
289 tensor(0.6514, device='cuda:0')
289 tensor(0.8327, device='cuda:0')
289 tensor(1.1587, device='cuda:0')
289 tensor(1.5618, device='cuda:0')
289 tensor(2.0076, device='cuda:0')
290 tensor(0.6519, device='cuda:0')
290 tensor(0.8321, device='cuda:0')
290 tensor(1.1589, device='cuda:0')
290 tensor(1.5637, device='cuda:0')
290 tensor(2.0129, device='cuda:0')
291 tensor(0.6534, device='cuda:0')
291 tensor(0.8344, device='cuda:0')
291 tensor(1.1632, device='cuda:0')
291 tensor(1.5718, device='cuda:0')
291 tensor(2.0207, device='cuda:0')
292 tensor(0.6488, device='cuda:0')
292 tensor(0.8285, device='cuda:0')
292 tensor(1.1581, device='cuda:0')
292 tensor(1.5618, device='cuda:0')
292 tensor(2.0119, device='c

...

1800 tensor(1.4897, device='cuda:0')
1801 tensor(0.6645, device='cuda:0')
1801 tensor(0.8470, device='cuda:0')
1801 tensor(1.0851, device='cuda:0')
1801 tensor(1.3064, device='cuda:0')
1801 tensor(1.4918, device='cuda:0')
1802 tensor(0.6657, device='cuda:0')
1802 tensor(0.8436, device='cuda:0')
1802 tensor(1.0833, device='cuda:0')
1802 tensor(1.3061, device='cuda:0')
1802 tensor(1.4929, device='cuda:0')
1803 tensor(0.6645, device='cuda:0')
1803 tensor(0.8468, device='cuda:0')
1803 tensor(1.0835, device='cuda:0')
1803 tensor(1.3056, device='cuda:0')
1803 tensor(1.4925, device='cuda:0')
1804 tensor(0.6660, device='cuda:0')
1804 tensor(0.8440, device='cuda:0')
1804 tensor(1.0839, device='cuda:0')
1804 tensor(1.3068, device='cuda:0')
1804 tensor(1.4931, device='cuda:0')
1805 tensor(0.6652, device='cuda:0')
1805 tensor(0.8485, device='cuda:0')
1805 tensor(1.0859, device='cuda:0')
1805 tensor(1.3073, device='cuda:0')
1805 tensor(1.4932, device='cuda:0')
1806 tensor(0.6663, device='cuda:0')
1

1861 tensor(0.6648, device='cuda:0')
1861 tensor(0.8496, device='cuda:0')
1861 tensor(1.0883, device='cuda:0')
1861 tensor(1.3036, device='cuda:0')
1861 tensor(1.4861, device='cuda:0')
1862 tensor(0.6666, device='cuda:0')
1862 tensor(0.8485, device='cuda:0')
1862 tensor(1.0853, device='cuda:0')
1862 tensor(1.3030, device='cuda:0')
1862 tensor(1.4849, device='cuda:0')
1863 tensor(0.6657, device='cuda:0')
1863 tensor(0.8512, device='cuda:0')
1863 tensor(1.0908, device='cuda:0')
1863 tensor(1.3050, device='cuda:0')
1863 tensor(1.4863, device='cuda:0')
1864 tensor(0.6655, device='cuda:0')
1864 tensor(0.8486, device='cuda:0')
1864 tensor(1.0845, device='cuda:0')
1864 tensor(1.3048, device='cuda:0')
1864 tensor(1.4880, device='cuda:0')
1865 tensor(0.6648, device='cuda:0')
1865 tensor(0.8488, device='cuda:0')
1865 tensor(1.0865, device='cuda:0')
1865 tensor(1.3041, device='cuda:0')
1865 tensor(1.4846, device='cuda:0')
1866 tensor(0.6651, device='cuda:0')
1866 tensor(0.8476, device='cuda:0')
1

1924 tensor(0.8502, device='cuda:0')
1924 tensor(1.0846, device='cuda:0')
1924 tensor(1.3058, device='cuda:0')
1924 tensor(1.4814, device='cuda:0')
1925 tensor(0.6680, device='cuda:0')
1925 tensor(0.8503, device='cuda:0')
1925 tensor(1.0892, device='cuda:0')
1925 tensor(1.3035, device='cuda:0')
1925 tensor(1.4856, device='cuda:0')
1926 tensor(0.6668, device='cuda:0')
1926 tensor(0.8476, device='cuda:0')
1926 tensor(1.0843, device='cuda:0')
1926 tensor(1.3025, device='cuda:0')
1926 tensor(1.4801, device='cuda:0')
1927 tensor(0.6657, device='cuda:0')
1927 tensor(0.8476, device='cuda:0')
1927 tensor(1.0840, device='cuda:0')
1927 tensor(1.2975, device='cuda:0')
1927 tensor(1.4797, device='cuda:0')
1928 tensor(0.6672, device='cuda:0')
1928 tensor(0.8513, device='cuda:0')
1928 tensor(1.0865, device='cuda:0')
1928 tensor(1.3036, device='cuda:0')
1928 tensor(1.4803, device='cuda:0')
1929 tensor(0.6656, device='cuda:0')
1929 tensor(0.8486, device='cuda:0')
1929 tensor(1.0842, device='cuda:0')
1

1987 tensor(1.2993, device='cuda:0')
1987 tensor(1.4738, device='cuda:0')
1988 tensor(0.6674, device='cuda:0')
1988 tensor(0.8505, device='cuda:0')
1988 tensor(1.0894, device='cuda:0')
1988 tensor(1.2974, device='cuda:0')
1988 tensor(1.4763, device='cuda:0')
1989 tensor(0.6645, device='cuda:0')
1989 tensor(0.8480, device='cuda:0')
1989 tensor(1.0865, device='cuda:0')
1989 tensor(1.3012, device='cuda:0')
1989 tensor(1.4764, device='cuda:0')
1990 tensor(0.6673, device='cuda:0')
1990 tensor(0.8488, device='cuda:0')
1990 tensor(1.0878, device='cuda:0')
1990 tensor(1.3004, device='cuda:0')
1990 tensor(1.4786, device='cuda:0')
1991 tensor(0.6647, device='cuda:0')
1991 tensor(0.8469, device='cuda:0')
1991 tensor(1.0855, device='cuda:0')
1991 tensor(1.2994, device='cuda:0')
1991 tensor(1.4750, device='cuda:0')
1992 tensor(0.6651, device='cuda:0')
1992 tensor(0.8488, device='cuda:0')
1992 tensor(1.0879, device='cuda:0')
1992 tensor(1.2994, device='cuda:0')
1992 tensor(1.4764, device='cuda:0')
1

tensor([[[[ 0.0096, -0.0059, -0.0063,  ..., -0.0237,  0.0043,  0.0005],
          [ 0.0082,  0.0072,  0.0015,  ...,  0.0143, -0.0057,  0.0055],
          [ 0.0073,  0.0004,  0.0004,  ...,  0.0098,  0.0026, -0.0083],
          ...,
          [ 0.0076,  0.0189,  0.0127,  ..., -0.0103,  0.0035,  0.0071],
          [-0.0046, -0.0030,  0.0163,  ...,  0.0112, -0.0197,  0.0217],
          [-0.0069,  0.0040, -0.0163,  ...,  0.0131, -0.0017, -0.0047]]]],
       device='cuda:0')

In [ ]:
# Generate image
# Put through encoder
# Put through simulator --> generating an average of output
# Generate loss